# Split Learning with CIFAR-10

### Setup
Install the required packages for training in the current Jupyter kernel

In [1]:
import sys
!{sys.executable} -m pip install -r ./requirements.txt

## 1. Download and split the CIFAR-10 dataset
To simulate a vertical split dataset, we first download the [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset and distribute it between the two clients, assuming an `OVERLAP` of 10,000 samples between the two clients' datasets.

In [2]:
%env SPLIT_DIR=/tmp/cifar10_vert_splits
%env OVERLAP=10000
!python3 ./cifar10_split_data_vertical.py --split_dir ${SPLIT_DIR} --overlap ${OVERLAP}

env: SPLIT_DIR=/tmp/cifar10_vert_splits
env: OVERLAP=10000
INFO:Cifar10VerticalDataSplitter:[identity=local, run=_]: Partition CIFAR-10 dataset into vertically with 10000 overlapping samples.
Files already downloaded and verified
INFO:Cifar10VerticalDataSplitter:[identity=local, run=_]: save /tmp/cifar10_vert_splits/overlap.npy
INFO:Cifar10VerticalDataSplitter:[identity=local, run=_]: save /tmp/cifar10_vert_splits/site-1.npy
INFO:Cifar10VerticalDataSplitter:[identity=local, run=_]: save /tmp/cifar10_vert_splits/site-2.npy


## 2. Run private set intersection
We are using NVFlare's FL simulator to run the following experiments.

In order to find the overlapping data indices between the different clients participating in split learning, 
we randomly select an subset of the training indices.

In [3]:
import os
from nvflare import SimulatorRunner

simulator = SimulatorRunner(
    job_folder=f"jobs/cifar10_psi",
    workspace="/tmp/nvflare/cifar10_psi",
    n_clients=2,
    threads=2
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

2023-03-13 18:32:11,301 - SimulatorRunner - INFO - Create the Simulator Server.
2023-03-13 18:32:11,305 - Cell - INFO - server: creating listener on tcp://0:56187
2023-03-13 18:32:11,307 - Cell - INFO - server: created backbone external listener for tcp://0:56187
2023-03-13 18:32:11,308 - ConnectorManager - INFO - 1997810: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2023-03-13 18:32:11,309 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE uds:///tmp/nvflare_371277] is starting
2023-03-13 18:32:11,811 - Cell - INFO - server: created backbone internal listener for uds:///tmp/nvflare_371277
2023-03-13 18:32:11,814 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:56187] is starting
2023-03-13 18:32:11,911 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 39529
2023-03-13 18:32:11,912 - SimulatorRunner - INFO - Deploy the Apps.
2023-03-13 18:32:11,921 - SimulatorRunner - INFO - 

The result will be saved on each client's working directory in `intersection.txt`.

We can check the correctness of the result by comparing to the generate ground truth overlap, saved in `overlap.npy`.

### Check the PSI result
We can check the correctness of the result by comparing to the generate ground truth overlap, saved in overlap.npy.

In [4]:
import os
import numpy as np

split_dir = os.environ["SPLIT_DIR"]
gt_overlap = np.load(os.path.join(split_dir, "overlap.npy"))

psi_overlap_1 = np.loadtxt("/tmp/nvflare/cifar10_psi/simulate_job/site-1/psi/intersection.txt")
psi_overlap_2 = np.loadtxt("/tmp/nvflare/cifar10_psi/simulate_job/site-2/psi/intersection.txt")
                     
print("gt_overlap", gt_overlap, f"n={len(gt_overlap)}")
print("psi_overlap_1", psi_overlap_1, f"n={len(psi_overlap_1)}")
print("psi_overlap_2", psi_overlap_2, f"n={len(psi_overlap_2)}")

intersect_1 = np.intersect1d(psi_overlap_1, gt_overlap, assume_unique=True)
intersect_2 = np.intersect1d(psi_overlap_2, gt_overlap, assume_unique=True)

print(f"Found {100*len(intersect_1)/len(gt_overlap):.1f}% of the overlapping sample ids for site-1.")
print(f"Found {100*len(intersect_2)/len(gt_overlap):.1f}% of the overlapping sample ids for site-2.")

gt_overlap [11841 19602 45519 ... 47278 37020  2217] n=10000
psi_overlap_1 [ 4481. 45431. 46253. ... 34846.   179.  7277.] n=10000
psi_overlap_2 [38639. 10733. 31911. ... 12172. 46167.   865.] n=10000
Found 100.0% of the overlapping sample ids for site-1.
Found 100.0% of the overlapping sample ids for site-2.


## 3. Run simulated split-learning experiments
Next we use the `intersection.txt` files to align the datasets on each participating site in order to do split learning.
The [config_fed_client.json](./jobs/cifar10_splitnn/site-1/config/config_fed_client.json) takes as input the previously generated intersection file for each site.
```
    {
        "id": "cifar10-learner",
        "path": "pt.learners.cifar10_learner_splitnn.CIFAR10LearnerSplitNN",
        "args": {
            "dataset_root": "{DATASET_ROOT}",
            "intersection_file": "{INTERSECTION_FILE}",
            "lr": 1e-2,
            "model": {"path": "pt.networks.split_nn.SplitNN", "args":  {"split_id":  0}},
            "timeit": true
        }
    }
```
To set the filename automatically, run:

In [5]:
!for i in {1..2}; \
do \
  CONFIG_FILE=jobs/cifar10_splitnn/site-${i}/config/config_fed_client.json; \
  INTERSECTION_FILE=/tmp/nvflare/cifar10_psi/simulate_job/site-${i}/psi/intersection.txt; \
  python3 ./set_intersection_file.py --config_file ${CONFIG_FILE} --intersection_file ${INTERSECTION_FILE}; \
done

Modified jobs/cifar10_splitnn/site-1/config/config_fed_client.json to use INTERSECTION_FILE=/tmp/nvflare/cifar10_psi/simulate_job/site-1/psi/intersection.txt
Modified jobs/cifar10_splitnn/site-2/config/config_fed_client.json to use INTERSECTION_FILE=/tmp/nvflare/cifar10_psi/simulate_job/site-2/psi/intersection.txt


To run the experiment, execute:

In [6]:
simulator = SimulatorRunner(
    job_folder=f"jobs/cifar10_splitnn",
    workspace="/tmp/nvflare/cifar10_splitnn",
    n_clients=2,
    threads=2
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

2023-03-13 18:33:14,062 - SimulatorRunner - INFO - Create the Simulator Server.
2023-03-13 18:33:14,066 - Cell - INFO - server: creating listener on tcp://0:47201
2023-03-13 18:33:14,068 - Cell - INFO - server: created backbone external listener for tcp://0:47201
2023-03-13 18:33:14,069 - ConnectorManager - INFO - 1998622: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2023-03-13 18:33:14,070 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE uds:///tmp/nvflare_920767] is starting
2023-03-13 18:33:14,573 - Cell - INFO - server: created backbone internal listener for uds:///tmp/nvflare_920767
2023-03-13 18:33:14,576 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:47201] is starting
2023-03-13 18:33:14,675 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 46447
2023-03-13 18:33:14,676 - SimulatorRunner - INFO - Deploy the Apps.
2023-03-13 18:33:14,680 - SimulatorRunner - INFO - 

The site containing the labels can compute accuracy and losses, which can be visualized in tensorboard.

In [7]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

%tensorboard --logdir /tmp/nvflare/cifar10_splitnn